In [1]:
import sys
import random
import json
import csv

sys.path.append('..')
from utils import SampleV3, DatasetV3

random.seed(10)

/home/local_nikhil/.conda/envs/mind_old/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
env.yml not found in /home/local_nikhil/Projects/mind!
Setting MODEL_ROOT="". Models will now be downloaded to conda env cache, if not already there
Other defaults are set to:
    DATA_DIR = "data"
    RESULTS_DIR = "results"
    HPARAMS_DIR = "hparams"


In [2]:
# Import csv file with first row as header
with open('../data/new_bigtom.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

characters = json.load(open('/home/local_nikhil/Projects/mind/data/synthetic_entities/actor.json'))

In [3]:
data[2]

{'story': '<character1> is a detective investigating a mysterious case. <character1> finds a crucial piece of evidence, a <container1> containing <state2>, at the crime scene. <character1> carefully places it in an evidence bag. Then <character1> finds another piece of evidence, a <container2> containing <state1>, and places it in a separate evidence bag. A rookie officer named <character2>, who is unaware of the importance of the evidence, accidentally swaps the contents of the <container1> with the <state1> while organizing the evidence.',
 'true_belief': '<character1> notices the swapped evidence.',
 'false_belief': '<character1> does not notice the swapped evidence.',
 'belief_question': 'Does <character1> believe the <container1> contains <state1>?',
 'worldstate_question': 'Does the <container1> contain <state1>?',
 'characters_1': 'Detective Jane, Detective Sarah, Detective Rachel, Detective Laura, Detective Kate',
 'characters_2': 'Officer Mike, Officer James, Officer Chris, Of

In [4]:
character_1, character_2 = random.choices(characters, k=2)

story = data[0]['story']
story += ' ' + data[0]['false_belief']
story += '\nQuestion: ' + data[0]['worldstate_question']
story = story.replace('<character1>', character_1)
story = story.replace('<character2>', character_2)
story = story.replace('<state2>', random.choice(data[0]['states_2'].split(', ')))
story = story.replace('<state1>', random.choice(data[0]['states_1'].split(', ')))
story = story.replace('<container1>', random.choice(data[0]['containers_1'].split(', ')))
story = story.replace('<container2>', random.choice(data[0]['containers_2'].split(', ')))

In [5]:
print(story)

Zoe is a botanist conducting research in a dense rainforest. Zoe wants to collect samples of a extraordinary shrub plant for her study. Zoe places the samples in a collection bag and labels it with extraordinary shrub. Then Zoe collects samples of a common fern plant and places them in a sample pouch. A fellow researcher named Kim, who believes the labels were mixed up, swaps the labels on the collection bag and sample pouch while Zoe is documenting her findings. Zoe does not notice the swapped labels.
Question: Does the collection bag contain common fern?


In [14]:
n_samples = 99
configs, samples = [], []

for idx in range(n_samples):
    protagonist, perpetrator = random.choices(characters, k=2)
    states = [random.choice(data[idx]['states_1'].split(', ')), random.choice(data[idx]['states_2'].split(', '))]
    containers = [random.choice(data[idx]['containers_1'].split(', ')), random.choice(data[idx]['containers_2'].split(', '))]
    obsr_event = data[idx]['false_belief']
    event_noticed = False

    config = SampleV3(
        story=data[idx]['story'],
        protagonist=protagonist,
        perpetrator=perpetrator,
        states=states,
        containers=containers,
        obsr_event=obsr_event,
        event_noticed=event_noticed
    )
    configs.append(config)

dataset = DatasetV3(configs)

for idx in range(n_samples):
    prompt, target = dataset.__getitem__(idx, question_type='true_state')
    samples.append({
        'prompt': prompt,
        'target': target
    })